In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from trajectory_visualization import plot_trajectory, sidebyside, shift_trajectory, overlay, shift_trajectory3D
from trajectory_visualization import plot_trajectories3D, plot_3Doverlay, plot_MSDorDeff, plot_MeanMSDorDeff, randtraj, multrandtraj
from trajectory_visualization import randtraj2, plot_Mean2DMSDsorDeff, plot_MSDorDeffLR, LRfor3D2D, fillin, prettify
from methods_of_Dcalc import Dpointder, Dlinalg, Dlinalgw
from Diffusion_Functions import frame_adjust, MSD_Deff_Calcs, prettify2, fillin2

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import scipy.optimize as opt
import scipy.stats as stat
from mpl_toolkits.mplot3d import Axes3D
from operator import itemgetter
import random
import numpy as np
import numpy.ma as ma
import numpy.linalg as la
from mpl_toolkits.mplot3d import Axes3D

pi = np.pi
sin = np.sin
cos = np.cos

In [2]:
#Currently, my code only has capability to do frame-by-frame analysis on 1 video.  Perhaps future functionality will
#be able to expand beyond this.  

name = 'confocal_1000ms_40x'

logplot = '{}_logplot'.format(name)
Mplot = '{}_Mplot'.format(name)
Dplot = '{}_Dplot'.format(name)
Hplot = '{}_Hplot'.format(name)
Hlogplot = '{}_Hlogplot'.format(name)
Cplot = '{}_Cplot'.format(name)

conversion = dict() #First element is the umppx, second is fps

conversion[1] = (1.24, 1.02, 1)
conversion[2] = (1.24, 1.02, 1)
conversion[3] = (1.24, 1.02, 1)
# conversion[4] = (0.30, 5.31, 1)
# conversion[5] = (0.30, 5.33, 1)
# conversion[6] = (0.30, 5.33, 1)
# conversion[7] = (0.30, 5.35, 1)

trajectory = dict()
cut = 5
great = 50

#Particles that you want manually filtered.
filtered = True
tofilt = np.array([35])

trajectory[1] = np.genfromtxt('../Frame-by-frame-analysis/confocal_1000ms_40x_1.csv',
            delimiter =",")
trajectory[2] = np.genfromtxt('../Frame-by-frame-analysis/confocal_1000ms_40x_2.csv',
            delimiter =",")
trajectory[3] = np.genfromtxt('../Frame-by-frame-analysis/confocal_1000ms_40x_3.csv',
            delimiter =",")

totvids = 3

for num in range(1, totvids + 1):
    
    #Remove titles from columns
    #trajectory[num]=np.delete(trajectory[num], 0,0)
    #Remove the number column from dataset
    trajectory[num]=np.delete(trajectory[num],0,1)

#Collect trajectories of separate videos into one array
#Comment this section out to make useable for a single trajectory
parts = dict()
tots = dict()
newtots = dict()
newtots[0] = 0
tlen = dict()
tlength = dict()
tlength[0] = 0

for num in range(1, totvids + 1):
    tots[num] = trajectory[num][-1, 0].astype(np.int64)
    parts[num] = tots[num]
    counter = 1
    newtots[num] = newtots[num-1] + tots[num]
    
    tlen[num] = trajectory[num].shape[0]
    tlength[num] = tlength[num-1] + tlen[num]

placeholder = np.zeros((tlength[totvids], 11))

#     while counter < parts[num]:
#         counter = counter + 1
#         placeholder = np.append(lehi[num], final[num][counter], axis=0)

for num in range(1, totvids + 1):
    placeholder[tlength[num-1]:tlength[num], :] = trajectory[num]
    placeholder[tlength[num-1]:tlength[num], 0] = placeholder[tlength[num-1]:tlength[num], 0] + newtots[num-1]

# trajectory[1] = placeholder

# trajectory[2][:,0] = trajectory[2][:,0] + tots[1]
# placeholder = np.append(placeholder, trajectory[2], axis=0)
# trajectory[3][:,0] = trajectory[3][:,0] + tots[2]
# placeholder = np.append(placeholder, trajectory[3], axis=0)

In [3]:
#Theoretical diffusion calculation using Stokes-Einstein

k = 1.380649 * 10**-23
TC = 37
TK = TC + 273
mu1 = 1.002*10**-3; mu2 = 0.798*10**-3; TC1 = 20; TC2 = 30

mu = (mu2 - mu1)*(TC - TC1)/(TC2 - TC1) + mu1
radius = (128/2) * 10**-9 #m
Deff1 = k*TK/(6*np.pi*mu*radius) #m2/s
Deff2 = Deff1 * 10**12 #um2/s
Deff2

5.414894389572004

In [4]:
def fillin2(data):
    """
    Fills in blanks of arrays without shifting frames by the starting frame.  Compare to fillin.
    """
    shap = int(max(data[:, 1])) + 1
    shape1 = int(min(data[:, 1]))
    newshap = shap - shape1
    filledin = np.zeros((newshap, 5))
    filledin[0, :] = data[0, :]

    count = 0
    new = 0
    other = 0
    tot = 0

    for num in range(1, newshap):
        if data[num-new, 1]-data[num-new-1, 1] == 1:
            count = count + 1
            filledin[num, :] = data[num-new, :]
        elif data[num - new, 1]-data[num - new - 1, 1] > 1:
            new = new + 1
            iba = int(data[num - new+1, 1]-data[num - new, 1])
            togoin = data[num - new]
            togoin[1] = togoin[1] + 1
            filledin[num, :] = togoin
            # dataset[2] = np.insert(dataset[2], [num + new - 2], togoin, axis=0)

        else:
            other = other + 1
        tot = count + new + other

    return filledin

In [5]:
rawframes = placeholder[:, 1]
#frames = np.linspace()
frames = np.linspace(min(rawframes),max(rawframes), max(rawframes)+1).astype(np.int64)
time = frames / conversion[1][1]

#Dictionaries including frames where particles aren't present.
x = dict()
y = dict()
z = dict()

#Dictionaries only including frames where particles are present.
xs = dict()
ys = dict()
zs = dict()

M1x = dict()
M1y = dict()
M1z = dict()
M2xy = dict()
M2xz = dict()
M2yz = dict()
M3 = dict()

#Shifted dictionaries
SM1x = dict()
SM1y = dict()
SM1z = dict()
SM2xy = dict()
SM2xz = dict()
SM2yz = dict()
SM3 = dict()

SD1x = dict()
SD1y = dict()
SD1z = dict()
SD2xy = dict()
SD2xz = dict()
SD2yz = dict()
SD3 = dict()

In [6]:
dataset = dict()

rawdataset = np.zeros(placeholder.shape)
particles = placeholder[:, 0]
total = int(max(particles))
total1 = total + 1
rawdataset = placeholder[:, :]

#Perform conversion on raw dataset
# rawdataset[:, 2:4] = conversion[1][0] * rawdataset[:, 2:4]
# rawdataset[:, 4] = conversion[1][2] * rawdataset[:, 4]

fixed = np.zeros(placeholder.shape)
fixed[:, 0:2] = rawdataset[:, 0:2]
fixed[:, 2:4] = conversion[1][0] * rawdataset[:, 2:4]
fixed[:, 4] = conversion[1][2] * rawdataset[:, 4]

# Creates an array for each trajectory containing all xyz data
for num in range(1, total1):

    hold = np.where(particles == num)
    itindex = hold[0]
    min1 = min(itindex)
    max1 = max(itindex)
    dataset[num] = (fixed[min1:max1+1, 0:5])

In [7]:
#I is a set of all nonzero elements in trajectories.
I = dict()
for num in range(1, total1):
    #Construct x, y, z
    dataset[num] = fillin2(dataset[num])
    x[num] = np.zeros(frames.shape[0])
    x[num][int(dataset[num][0,1]):int(dataset[num][-1,1])+1] = dataset[num][:, 2]
    y[num] = np.zeros(frames.shape[0])
    y[num][int(dataset[num][0,1]):int(dataset[num][-1,1])+1] = dataset[num][:, 3]
    z[num] = np.zeros(frames.shape[0])
    z[num][int(dataset[num][0,1]):int(dataset[num][-1,1])+1] = dataset[num][:, 4]
    
    xs[num] = np.zeros(frames.shape[0])
    xs[num][0:int(dataset[num][-1,1])+1-int(dataset[num][0,1])] = dataset[num][:, 2]
    ys[num] = np.zeros(frames.shape[0])
    ys[num][0:int(dataset[num][-1,1])+1-int(dataset[num][0,1])] = dataset[num][:, 3]
    zs[num] = np.zeros(frames.shape[0])
    zs[num][0:int(dataset[num][-1,1])+1-int(dataset[num][0,1])] = dataset[num][:, 4]

In [8]:
x[742]

array([  0.     ,   0.     ,   0.     ,   0.     ,   0.     ,   0.     ,
         0.     ,   0.     ,   0.     ,   0.     ,   0.     ,   0.     ,
         0.     ,   0.     ,   0.     ,   0.     ,   0.     ,   0.     ,
         0.     ,   0.     ,   0.     ,   0.     ,   0.     ,   0.     ,
         0.     ,   0.     ,   0.     ,   0.     ,   0.     ,   0.     ,
         0.     ,   0.     ,   0.     ,   0.     ,   0.     ,   0.     ,
         0.     ,   0.     ,   0.     ,   0.     ,   0.     ,   0.     ,
        57.60916,  57.71332,   0.     ,   0.     ])

In [9]:
#I wrote a short bit of code to filter out trajectories that are too short to be included.  This also reduces computing
#time by cutting down on MSDs that need to be calculated.  

cutoff = cut
x1 = dict()
y1 = dict()
z1 = dict()

xs1 = dict()
ys1 = dict()
zs1 = dict()

fifties = 0
nones = 0

for num in range(1, total1):
    if np.count_nonzero(x[num]) < cutoff:
        nones = nones + 1
    else:
        fifties = fifties + 1
        x1[num - nones] = x[num]
        y1[num - nones] = y[num]
        z1[num - nones] = z[num]

        xs1[num - nones] = xs[num]
        ys1[num - nones] = ys[num]
        zs1[num - nones] = zs[num]
        # I must also redefine the particle numbers to reflect the new set.
        #teancum[num - nones][:, 0] = fifties

#fifties is the new total1 after filtering out short trajectories
total1 = fifties + 1
fifties

542

In [11]:
xs1[500]

array([ 295.37048,  295.43992,  295.43992,  295.24772,  295.44612,
          0.     ,    0.     ,    0.     ,    0.     ,    0.     ,
          0.     ,    0.     ,    0.     ,    0.     ,    0.     ,
          0.     ,    0.     ,    0.     ,    0.     ,    0.     ,
          0.     ,    0.     ,    0.     ,    0.     ,    0.     ,
          0.     ,    0.     ,    0.     ,    0.     ,    0.     ,
          0.     ,    0.     ,    0.     ,    0.     ,    0.     ,
          0.     ,    0.     ,    0.     ,    0.     ,    0.     ,
          0.     ,    0.     ,    0.     ,    0.     ,    0.     ,    0.     ])

In [ ]:
x[511]

In [ ]:
total1